In [1]:
import os
import random
import argparse
import glob
import csv
import subprocess

In [2]:
hcp_list = '/Users/emmatosato/Documents/PhD/ANM_Verona/data_utils/list_HCP.txt'
n_augmentations = 10
subset_size = 17

In [3]:
# Read HCP file
with open(hcp_list, "r") as f:
    hcp_pool = [line.strip() for line in f.readlines()]

random.seed(42)

# Generate fixed HCP subsets that NOT overlap
shuffled_hcp = random.sample(hcp_pool, n_augmentations * subset_size)

hcp_subsets = [
    shuffled_hcp[i * subset_size : (i + 1) * subset_size]
    for i in range(n_augmentations)
]

In [5]:
hcp_subsets

[['901139',
  '145834',
  '109123',
  '191336',
  '180533',
  '177645',
  '155938',
  '140117',
  '725751',
  '134627',
  '818859',
  '381038',
  '114823',
  '111514',
  '134829',
  '176542',
  '178142'],
 ['601127',
  '833249',
  '943862',
  '757764',
  '169747',
  '901442',
  '380036',
  '177140',
  '401422',
  '191841',
  '102311',
  '159239',
  '898176',
  '209228',
  '771354',
  '158136',
  '871762'],
 ['205220',
  '905147',
  '872764',
  '257845',
  '135124',
  '221319',
  '212419',
  '186949',
  '116726',
  '429040',
  '146735',
  '251833',
  '115825',
  '942658',
  '157336',
  '365343',
  '200210'],
 ['199655',
  '397760',
  '389357',
  '167440',
  '192641',
  '671855',
  '214524',
  '878877',
  '108323',
  '203418',
  '146129',
  '263436',
  '550439',
  '572045',
  '393247',
  '118225',
  '169343'],
 ['910241',
  '177746',
  '200311',
  '436845',
  '131722',
  '169040',
  '167036',
  '706040',
  '204521',
  '150423',
  '214019',
  '751550',
  '200614',
  '115017',
  '197348',


In [4]:
# Flatten all subsets into a single list
all_hcps = [hcp for subset in hcp_subsets for hcp in subset]

# Compare total length vs. number of unique HCPs
if len(all_hcps) == len(set(all_hcps)):
    print("Subsets are disjoint: no overlaps.")
else:
    print("Warning: there are overlapping HCPs between subsets.")


Subsets are disjoint: no overlaps.
